In [13]:
import pandas as pd
import numpy as np
import ta 
import math


#dados foram baixados de https://www.cryptodatadownload.com/ 
#indicadores tecnicos de https://www.akademiabaru.com/doc/ARBMSV14_N1_P35_41.pdf
#indicadores são:  MACD, RSI, SO, MA, EMA, ROC e VT.


df = pd.read_csv("Binance_BTCUSDT_minute.csv",skiprows=1,nrows=100) #lendo somente as 10000 primeiras linhas para salvar processamento enquanto o codigo é desenvolvido
#lembrar de remover ultimas 74 rows


#convertendo colunas numericas do dataframe para os tipos corretos
keys_ = [i for i in df.keys()][3:9]
for key in keys_:
    pd.to_numeric(df[key])

#calculando os indicadores


#1 - MACD 
df["MACD"] = ta.trend.macd(close=df.close) # Nas primeiras 26 linhas MACD retorna nan, pois precisa de 26 elementos.

#2 - RSI lembrar de ajustar window para melhorar resultados. Lembrar que nas primeiras 14 linhas RSI = nan, pois ele precisa de 14 elementos anteriores.
RSI_window = 14
df["RSI"] = ta.momentum.rsi(df.close,window=RSI_window)


#3 - stochastic oscillator (SO)
SO_window = 14
df["SO"] = ta.momentum.stoch(close=df.close,high=df.high,low=df.low,window=SO_window)

#3 - Simple moving average (SMA)

df["12_SMA"] = ta.trend.sma_indicator(close=df.close,window=12)
df["26_SMA"] = ta.trend.sma_indicator(close=df.close,window=26)

#4 - Exponential moving average (EMA)

df["12_EMA"] = ta.trend.ema_indicator(close=df.close,window=12)
df["26_EMA"] = ta.trend.ema_indicator(close=df.close,window=26)

#5 - Rate of change (ROC)
ROC_window = 24 #definir window direito (minima ideia qual window tem que meter aqui)
df["ROC"] = ta.momentum.roc(close=df.close,window=ROC_window)


#6 - indicadores de volume, mineração de btc e remuneração dos mineradores

# adicionando coluna que possui 0, caso o ultimo close seja > que o close atual e 1 caso o contrário

change = [0]
for i in range(1,len(df)):
    diff = df.iloc[i].close- df.iloc[i-1].close
    if diff >= 0 :
        change.append(1)
    if diff < 0:
        change.append(0)

df["change"] = change
df

#logit(p)= ln(p/1-p) = bo + b1*MACD + b2*RSI + b3*SO + b4*MA +b5*EMA + b6*ROC + b7*VT

   






    


,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount,MACD,RSI,SO,12_SMA,26_SMA,12_EMA,26_EMA,ROC,change
0,1636941960000,2021-11-15 02:06:00,BTC/USDT,65861.12,65861.68,65861.12,65861.68,0.01732,1.140721e+03,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1636941900000,2021-11-15 02:05:00,BTC/USDT,65816.13,65861.68,65816.13,65861.13,9.91196,6.526023e+05,506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1636941840000,2021-11-15 02:04:00,BTC/USDT,65861.68,65869.00,65816.13,65818.90,11.01774,7.254315e+05,526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1636941780000,2021-11-15 02:03:00,BTC/USDT,65868.06,65875.00,65851.84,65861.68,11.55151,7.608195e+05,544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1636941720000,2021-11-15 02:02:00,BTC/USDT,65924.34,65924.82,65856.42,65868.06,28.84769,1.900776e+06,809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1636936260000,2021-11-15 00:31:00,BTC/USDT,66037.95,66066.09,66000.64,66041.71,66.36198,4.381815e+06,1494,31.575057,56.455891,69.644497,65975.703333,65994.083846,65997.863489,65966.288432,0.063167,0
96,1636936200000,2021-11-15 00:30:00,BTC/USDT,65944.90,66051.66,65944.89,66037.96,47.55047,3.138492e+06,1411,32.434746,56.106373,68.414666,65990.605833,65995.651923,66004.032183,65971.597437,0.039826,0
97,1636936140000,2021-11-15 00:29:00,BTC/USDT,65909.50,65998.04,65909.50,65944.89,52.96210,3.492982e+06,1572,25.314277,48.140493,37.891906,65995.679167,65993.531538,65994.933386,65969.619109,-0.190130,0
98,1636936080000,2021-11-15 00:28:00,BTC/USDT,65895.91,65928.35,65877.91,65909.50,34.38062,2.266250e+06,1356,16.623947,45.495380,26.285583,65992.259167,65989.601923,65981.789788,65965.165841,-0.260964,0
